<a href="https://colab.research.google.com/github/myielin/dataMiningUTFPR/blob/master/dataMiningUTFPR/ZosteraMarinaRNA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pré-processamento com MathFeature


[Repositório MathFeature](https://bonidia.github.io/MathFeature/)
<br>
[Repo do notebook](https://github.com/myielin/dataMiningUTFPR/geneFeatures/)


### MathFeature Preprocssing:

Os scripts do framework foram executados localmente nas sequências fasta 

(linha de comando, na pasta MathFeature/preprocessing)


```
python3 count_sequences.py -i ../[PATH]/Zmarina-lncRNA.fasta
```



- número de sequências em mRNA: 21483
- número de sequências em lncRNA: 1393

Por haver uma diferença muito grande nas quantidades de sequências, o primeiro arquivo deve ser reduzido proporcionalmente ao segundo.
<br>
Para isso, no próximo exemplo é criado um novo arquivo fasta com a amostra necessária utilizando o script sampling.py



```
python3 sampling.py -i ../[PATH]/gencode.v42.transcripts.fa -o ../[PATH]/samp-mRNA.fa -p 1393
```

Em seguida, são gerados mais dois arquivos com o pré-processamento do mRNA e lncRNA, onde são removidos os ruídos referentes a caracteres que não correspondem a nucleotídeos:



```
python3 preprocessing.py -i ../[PATH]/samp-mRNA.fa -o ../[PATH]/prep-mRNA.fa
```


```
python3 preprocessing.py -i ../[PATH]/samp-mRNA.fa -o ../[PATH]/prep-lncRNA.fa
```







### Mathfeature Methods:

No exemplo serão utilizados os métodos kmer, transformada de Fourier e entropia de Shannon. Cada método gera um arquivo de csv com base nas informações processadas em fasta

Kmer Args:
- l: label - corresponde a como a sequencia de entrada será representada. Aqui será usado 0 para RNA não codificante, pois a busca será por RNA mensageiro (1)
- t: tipo de extração
- seq: RNA (1) ou DNA 

```
python3 ExtractionTechniques.py -i ../[PATH]/prep-lncRNA.fa -o ../[PATH]/kmer-lncRNA.csv -l 0 -t kmer -seq 1
```

```
python3 ExtractionTechniques.py -i ../[PATH]/prep-mRNA.fa -o ../[PATH]/kmer-mRNA.csv -l 1 -t kmer -seq 1
```





Shannon Entropy Args:
- k: tamanho do kmer que é utilizado nas medições de entropia 
- e: tipo de entropia (Shannon ou Tsallis)

```
python3 EntropyClass.py -i ../[PATH]/prep-lncRNA.fa -o ../[PATH]/shannon-lncRNA.csv -l 0 -k 3 -e Shannon
```

```
python3 EntropyClass.py -i ../[PATH]/prep-mRNA.fa -o ../[PATH]/shannon-mRNA.csv -l 1 -k 3 -e Shannon
```





Transformação de Fourier 


```
python3 FourierClass.py -i ../[PATH]/prep-lncRNA.fa -o ../[PATH]/fourier-lncRNA.csv -l 0 -r 3
```

```
python3 FourierClass.py -i ../[PATH]/prep-mRNA.fa -o ../[PATH]/fourier-mRNA.csv -l 1 -r 3
```


Os arquivos csv das diferentes classes de RNA foram então concatenados, gerando os arquivos específicos para lncRNA e mRNA contendo as 3 técnicas.


(na pasta onde estão localizados os arquivos csv)
```
python3 MathFeature/preprocessing/concatenate.py -n 3 -o lncRNA.csv

```
(a ordem de concatenação dos conjuntos foi: kmer, fourier, shannon)



### pré-processamento nos conjuntos gerados

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split as tts
from sklearn.preprocessing import StandardScaler as scalar

In [2]:
mrna = pd.read_csv("https://raw.githubusercontent.com/myielin/dataMiningUTFPR/master/geneFeatures/mRNA.csv")
lrna = pd.read_csv("https://raw.githubusercontent.com/myielin/dataMiningUTFPR/master/geneFeatures/lncRNA.csv")

Alguns valores gerados foram NaN, e serão preenchidos utilizando o método ffill ou dropados <br> <br>
Os conjuntos de mRNA e lncRNA serão concatenados em uma base geral de rna

In [3]:
lrna.shape

(1393, 108)

In [4]:
rna = pd.concat([lrna, mrna])

# dropando a linha onde a concatenação gera strings com os nomes dos atributos
rna.drop(1393, axis=0, inplace=True)

In [5]:
rna.head()

,nameseq,A,C,G,T,AA,AC,AG,AT,CA,...,variance,interquartile_range,semi_interquartile_range,coefficient_of_variation,skewness,kurtosis,k1,k2,k3,label
0,lcl|Zmarina_Zosma100g00100.1,0.241546,0.231884,0.251208,0.275362,0.084746,0.043584,0.050847,0.060533,0.05569,...,3.013191e+05,632.721834,316.360917,1.031537,1.125760,0.256069,1.997020,3.950788,5.812913,0
1,lcl|Zmarina_Zosma101g00060.1,0.352727,0.183636,0.2,0.263636,0.151184,0.047359,0.083789,0.069217,0.071038,...,7.884508e+05,900.300605,450.150302,1.087848,0.881249,0.259474,1.950744,3.841459,5.677286,0
2,lcl|Zmarina_Zosma101g00070.1,0.250685,0.194521,0.208219,0.346575,0.0631,0.041152,0.039781,0.105624,0.049383,...,1.208722e+06,1100.284849,550.142424,1.041883,0.929857,0.250195,1.961041,3.889054,5.774241,0
3,lcl|Zmarina_Zosma101g00100.1,0.300866,0.212121,0.151515,0.335498,0.104121,0.047722,0.056399,0.093275,0.073753,...,5.747894e+05,776.830487,388.415243,1.076514,0.955633,0.251114,1.936980,3.849014,5.650810,0
4,lcl|Zmarina_Zosma101g00360.1,0.357488,0.202899,0.154589,0.285024,0.150485,0.072816,0.038835,0.092233,0.067961,...,9.021670e+04,427.597833,213.798917,0.942988,1.230547,0.289432,1.929947,3.804582,5.464826,0


Os demais valores categóricos precisam ser transformados em numéricos para então serem divididos os conjuntos de treino e teste

In [6]:
rna.fillna(method="ffill", inplace=True)
#rna.dropna(inplace=True)

rna.drop('nameseq', axis=1, inplace = True)

# valores do tipo int e str estão misturados nas variáveis, então devem ser convertidos em numéricos
for i in (rna.columns):
  rna[i] = rna[i].apply(lambda k: float(k))

In [7]:
# separando os conjuntos x e y 
y = rna['label']
x = rna.drop('label', axis=1)

Os modelos serão testados de três formas:
- sem normalização (sem rodar a célula seguinte)
- normalização padrão
- escala MinMax

In [8]:
norm = scalar()      # método de padronização (minMax ou normalização)
norm.fit_transform(x)

array([[-1.11152078,  0.71114766,  0.60703808, ...,  1.08572238,
         0.87407965,  0.44203047],
       [ 1.3611845 , -0.49562657, -0.73795147, ..., -0.49709104,
        -0.76016962, -0.54054464],
       [-0.908266  , -0.2233907 , -0.52207178, ..., -0.1449098 ,
        -0.04871886,  0.16186501],
       ...,
       [-0.03219004, -0.01845147, -0.17655442, ...,  0.27471867,
         0.394394  ,  0.46372257],
       [ 1.15652735, -0.60517361, -1.23554465, ...,  0.27471867,
         0.394394  ,  0.46372257],
       [ 0.42603264, -0.59629293,  1.02157278, ...,  0.27471867,
         0.394394  ,  0.46372257]])

In [9]:
x_train, x_test, y_train, y_test = tts(x, y, test_size = 0.25)

# Modelos preditivos
O modelo de maior enfoque para análise será árvore de decisão, mas os demais serão utilizados para demonstração.

In [10]:
from sklearn.metrics import (recall_score,
                             accuracy_score,
                             precision_score,
                             f1_score)

In [11]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier as Knn

In [12]:
dtc = DecisionTreeClassifier()

dtc.fit(x_train, y_train)
pred = dtc.predict(x_test)

print("acuracidade: ", round(accuracy_score(y_test, pred)*100, 2))
print("Precisão: ", round(precision_score(y_test, pred)*100, 2))
print("Recall: ", round(recall_score(y_test, pred)*100, 2))
print("F1: ", round(f1_score(y_test, pred)*100, 2))

acuracidade:  99.9
Precisão:  100.0
Recall:  99.72
F1:  99.86


obs: O genoma da planta Zostera Marina é relativamente pequeno quando comparado ao de animais, então a base de dados é consideravelmente menor do que as presentes no Gencode. Talvez esse fator, somado ao nivelamento dos arquivos FASTA, possa explicar a razão para valores de acuracidade tão altos independentemente da normalização.

In [13]:
knn = Knn(n_neighbors=3)

knn.fit(x_train, y_train)
pred = knn.predict(x_test)

precisao = round(precision_score(y_test, pred)*100, 2)
print(precisao)

99.72


In [14]:
nb = GaussianNB()

nb.fit(x_train, y_train)
pred = nb.predict(x_test)

precisao = round(accuracy_score(y_test, pred)*100, 2)
print(precisao)

99.52


# Feature Importance
Será utilizado o método wrapper com Recursive feature elimination para análise do algorítmo decision tree. 


In [15]:
from sklearn.feature_selection import RFE

In [ ]:
selector = RFE(dtc, step=1).fit(x, y)
print(selector.ranking_)

feat = selector.fit_transform(x,y)
print(feat)

In [ ]:
temp = pd.Series(selector.support_,index = x.columns)
wrapper = temp[temp==True].index

feat_list = wrapper.tolist()
feat_list

In [18]:
feat_tree = DecisionTreeClassifier()
feat_tree.fit(feat, y)

imp_tree = feat_tree.feature_importances_

for i,j in enumerate(imp_tree):
  if j != 0:
    print('Feature %s - score %f' % (feat_list[i], j) )

Feature TAG - score 0.000538
Feature minimum - score 0.999462


A maior parte das características teve importância atribuída igual a zero, e os resultados foram definidos com base no atributo Feature minimum, do modelo de transformada de Fourier

Utilizando o método filter as características determinantes serão mantidas no modelo, enquanto as demais serão removidas:

In [19]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif

In [20]:
selection = SelectKBest(score_func=mutual_info_classif, k=2)
selection.fit(x_train, y_train)

sel_cols = selection.get_support(indices=True)
sel_cols

array([87, 93])

In [22]:
xsel_treino = selection.transform(x_train)
xsel_teste = selection.transform(x_test)

sel_model = DecisionTreeClassifier()
sel_model.fit(xsel_treino, y_train)

sel_pred = sel_model.predict(xsel_teste)
round(accuracy_score(y_test,sel_pred)*100,2)

100.0